In [211]:
import tensorflow as tf
import numpy as np
import pandas as pd
from keras import layers
import keras

In [245]:
def model(shape1, shape2, shape3):
    x1 = layers.Input(shape=shape1)
    x2 = layers.Input(shape=shape2)
    x3 = layers.Input(shape=shape3)
    embedding1 = layers.Embedding(3, 2, mask_zero=True, input_length=1)
    embedding2 = layers.Embedding(3, 2, mask_zero=True, input_length=1)
    concat = layers.Concatenate(name='Input_merging', axis=len(x3.shape)-1)

    masking_layer = layers.Masking()

    # layer_cnn = layers.Conv1D(filters=6, kernel_size=3, padding='same', name='cnn')
    
    # dense = layers.Dense(units=2, activation='softmax')

    e1 = embedding1(x1)
    e2 = embedding2(x2)
    c = concat([e1, e2, x3])
    m = masking_layer(c)
    print(m._keras_mask)
    # cnn_output = layer_cnn(c)
    # y = dense(cnn_output)
    # output = tf.squeeze(y)
    model = keras.Model([x1, x2, x3], e1)
    # model = keras.Model([x1, x2, x3], output)
    return model

model = model((3, 1), (3, 1), (3, 1, 2))   # (sequence_len, dimension)
model.summary()

KerasTensor(type_spec=TensorSpec(shape=(None, 3, 1), dtype=tf.bool, name=None), name='masking_15/Squeeze:0')
Model: "model_69"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_178 (InputLayer)          [(None, 3, 1)]       0                                            
__________________________________________________________________________________________________
input_179 (InputLayer)          [(None, 3, 1)]       0                                            
__________________________________________________________________________________________________
input_180 (InputLayer)          [(None, 3, 1, 2)]    0                                            
__________________________________________________________________________________________________
embedding_148 (Embedding)       (None, 3, 1, 2)      6           input_178[0][0] 

In [246]:
model.layers[3].get_weights()

[array([[-0.04509665,  0.02554139],
        [-0.02233099, -0.01081265],
        [-0.00100253,  0.03505286]], dtype=float32)]

In [247]:
# input_array1 = tf.convert_to_tensor([[[1], [1], [0]], [[2], [0], [1]], [[1], [4], [1]], [[4], [3], [2]]])
# input_array2 = tf.convert_to_tensor([[[1], [1], [0]], [[2], [0], [1]], [[1], [4], [1]], [[4], [3], [2]]])
# input_array3 = tf.convert_to_tensor([[[[1, 1]], [[1, 1]], [[0, 0]]], [[[2, 1]], [[2, 0]], [[2, 2]]], [[[4, 1]], [[1, 1]], [[2, 0]]], [[[3, 1]], [[2, 2]], [[0, 1]]]])

input_array1 = tf.convert_to_tensor([[[1], [1], [0]], [[2], [0], [1]]])
input_array2 = tf.convert_to_tensor([[[1], [1], [0]], [[2], [0], [1]]])
input_array3 = tf.convert_to_tensor([[[[1, 1]], [[1, 1]], [[0, 0]]], [[[2, 1]], [[0, 0]], [[2, 2]]]])


model.compile('rmsprop', 'mse')
output_array = model.predict([input_array1, input_array2, input_array3])
output_array

array([[[[-0.02233099, -0.01081265]],

        [[-0.02233099, -0.01081265]],

        [[-0.04509665,  0.02554139]]],


       [[[-0.00100253,  0.03505286]],

        [[-0.04509665,  0.02554139]],

        [[-0.02233099, -0.01081265]]]], dtype=float32)

In [601]:
input_array1 = tf.convert_to_tensor([[[1], [2], [1], [0]], [[1], [0], [0], [2]]])
input_array2 = tf.convert_to_tensor([[[1], [2], [2], [0]], [[1], [0], [0], [2]]])
input_array3 = tf.convert_to_tensor([[[[1, 1]], [[2, 2]], [[1, 0]], [[0, 0]]], [[[1, 1]], [[0, 0]], [[0, 0]], [[2, 2]]]])

In [742]:
print(input_array1.shape)
print(input_array2.shape)
print(input_array3.shape)

(2, 4, 1)
(2, 4, 1)
(2, 4, 1, 2)


In [602]:
from model import CNN_with_mask

In [603]:
masking_layer = layers.Masking(mask_value=0)
embedding1 = layers.Embedding(3, 2, mask_zero=True, input_length=1)
concat = layers.Concatenate(name='Input_merging')
layer_cnn = CNN_with_mask(filters=6, kernel_size=3, name='cnn')
dense = layers.Dense(units=2, activation='softmax')

e = embedding1(input_array1)

unmasked_embedding = tf.cast(input_array3, tf.float32)
masked_embedding = masking_layer(unmasked_embedding)
c = concat([e, masked_embedding])
cnn = layer_cnn(c)
output = dense(cnn)
print(output._keras_mask)

print(output)

tf.Tensor(
[[[ True]
  [ True]
  [ True]
  [False]]

 [[ True]
  [False]
  [False]
  [ True]]], shape=(2, 4, 1), dtype=bool)
tf.Tensor(
[[[[0.3363277  0.66367227]]

  [[0.20549262 0.7945073 ]]

  [[0.35759407 0.6424059 ]]

  [[0.50300574 0.4969942 ]]]


 [[[0.3363277  0.66367227]]

  [[0.50300574 0.4969942 ]]

  [[0.50300574 0.4969942 ]]

  [[0.20549262 0.7945073 ]]]], shape=(2, 4, 1, 2), dtype=float32)


In [680]:
label = np.array([[[[1.,  0.]],
  [[0.,  1.]],
  [[1., 0.]],
  [[0., 0.]]],
 [[[0.,  1.]],
  [[0., 0.]],
  [[0., 0.]],
  [[0.,  1.]]]])

In [744]:
label.shape

(2, 4, 1, 2)

In [718]:
input_tensor1 = keras.Input(shape=(4, 1))
input_tensor2 = keras.Input(shape=(4, 1, 2))

masking_layer = layers.Masking(mask_value=0)
embedding1 = layers.Embedding(3, 2, mask_zero=True, input_length=1)
concat = layers.Concatenate(name='Input_merging')
layer_cnn = CNN_with_mask(filters=6, kernel_size=3, name='cnn')
dense = layers.Dense(units=2, activation='softmax')

e = embedding1(input_tensor1)

unmasked_embedding = tf.cast(input_tensor2, tf.float32)
masked_embedding = masking_layer(unmasked_embedding)
c = concat([e, masked_embedding])
cnn = layer_cnn(c)
output = dense(cnn)

model = keras.Model([input_tensor1, input_tensor2], output)


In [720]:
model.compile(loss=tf.keras.losses.MeanSquaredError(), optimizer='adam', metrics=[keras.metrics.BinaryAccuracy()])
model.evaluate([input_array1, input_array3], label)

1/1 [==============================] - 0s 316ms/step - loss: 0.1241 - binary_accuracy: 0.6000


[0.12414862960577011, 0.6000000238418579]

In [739]:
def model(input_dim, seq_len, embed_space_size, embed_action_size):
    input_space = keras.Input(shape=(seq_len, input_dim[0]), name='Space_input')
    input_action = keras.Input(shape=(seq_len, input_dim[1]), name='Action_input')
    input_others = keras.Input(shape=(seq_len, input_dim[0], input_dim[2]), name='Other_input')

    space_embedding = layers.Embedding(input_dim=embed_space_size, output_dim=4, mask_zero=True, name='Space_embedding')
    action_embeding = layers.Embedding(input_dim=embed_action_size, output_dim=4, mask_zero=True, name='Action_embedding')

    masking_layer = layers.Masking(mask_value=0)
    
    input_concat_embedding = layers.Concatenate(name='Input_merging')
    
    layer_cnn = CNN_with_mask(filters=4+4+input_dim[2], kernel_size=3, name='cnn')
    layer_dense = layers.Dense(units=3, activation='softmax')

    '''define forward'''
    inputs = [input_space, input_action, input_others]
    embeded_space = space_embedding(input_space)
    embeded_action = action_embeding(input_action)

    unmasked_others = tf.cast(input_others, tf.float32)
    masked_others = masking_layer(unmasked_others)

    embeded_input = input_concat_embedding([embeded_space, embeded_action, masked_others])

    cnn = layer_cnn(embeded_input)
    output_prob = layer_dense(cnn)

    model = keras.Model(inputs=inputs, outputs=output_prob, name='classifier')

    return model

In [740]:
model = model((1, 1, 7), 3, 5, 5)
model.summary()

Model: "classifier"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Other_input (InputLayer)        [(None, 3, 1, 7)]    0                                            
__________________________________________________________________________________________________
Space_input (InputLayer)        [(None, 3, 1)]       0                                            
__________________________________________________________________________________________________
Action_input (InputLayer)       [(None, 3, 1)]       0                                            
__________________________________________________________________________________________________
tf.cast_67 (TFOpLambda)         (None, 3, 1, 7)      0           Other_input[0][0]                
_________________________________________________________________________________________

In [115]:
def model(input_shape):
    x = layers.Input(shape=input_shape)
    embedding = layers.Embedding(5, 2, mask_zero=True, input_length=1)
    # flatten = layers.Flatten()
    
    dense = layers.Dense(units=2, activation='softmax')
    
    reshape = layers.Reshape((2, 2))

    e = embedding(x)
    y = dense(e)
    print(y.shape)
    model = keras.Model(x, reshape(y))
    return model

model = model((2, 1))   # (sequence_len, dimension)
model.summary()

(None, 2, 1, 2)
Model: "model_26"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_43 (InputLayer)        [(None, 2, 1)]            0         
_________________________________________________________________
embedding_44 (Embedding)     (None, 2, 1, 2)           10        
_________________________________________________________________
dense_29 (Dense)             (None, 2, 1, 2)           6         
_________________________________________________________________
reshape_8 (Reshape)          (None, 2, 2)              0         
Total params: 16
Trainable params: 16
Non-trainable params: 0
_________________________________________________________________


In [116]:
input_array = [[[1], [2]], [[2], [0]], [[4], [1]], [[3], [2]]]


model.compile('rmsprop', 'mse')
output_array = model.predict(input_array)
output_array

array([[[0.52306634, 0.4769337 ],
        [0.47747383, 0.52252626]],

       [[0.47747383, 0.52252626],
        [0.5073446 , 0.49265543]],

       [[0.48104572, 0.51895434],
        [0.52306634, 0.4769337 ]],

       [[0.5001904 , 0.49980965],
        [0.47747383, 0.52252626]]], dtype=float32)

---

In [117]:
def model(input_shape):
    x = layers.Input(shape=(input_shape,))

    embedding = layers.Embedding(5, 2, mask_zero=True, input_length=1)
    flatten = layers.Flatten()
    dense = layers.Dense(units=1, activation='softmax')
    
    e = embedding(x)
    p = flatten(e)
    y = dense(p)
    model = keras.Model(x, y)
    return model

In [118]:
m = model(1)
m.summary()

Model: "model_27"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_44 (InputLayer)        [(None, 1)]               0         
_________________________________________________________________
embedding_45 (Embedding)     (None, 1, 2)              10        
_________________________________________________________________
flatten_7 (Flatten)          (None, 2)                 0         
_________________________________________________________________
dense_30 (Dense)             (None, 1)                 3         
Total params: 13
Trainable params: 13
Non-trainable params: 0
_________________________________________________________________


In [119]:
model = keras.models.Sequential()
model.add(layers.Embedding(5, 2))

input_array = [[1], [2], [3]]

model.compile('rmsprop', 'mse')
output_array = model.predict(input_array)

In [120]:
print(input_array)
print(output_array)

[[1], [2], [3]]
[[[ 0.02308043 -0.03748535]]

 [[-0.0021935   0.01627899]]

 [[ 0.03740347  0.03914301]]]


In [121]:
output_array.shape

(3, 1, 2)

In [122]:
output_array.resize([output_array.shape[0], output_array.shape[2]])

In [123]:
output_array

array([[ 0.02308043, -0.03748535],
       [-0.0021935 ,  0.01627899],
       [ 0.03740347,  0.03914301]], dtype=float32)